<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON611
### Lecture 8 -  SQL - intermediate and intro to advanced topics.
- Notes adapted from: 

    1. [SQL for Data Science: Presentation by Renée Teate](https://docs.google.com/presentation/d/1qOtT2M-rjYa9JzKNz6_CxvfbihmqcmG9Ax6LgIriz0Y/mobilepresent?slide=id.p)
    2. [Mode Analytics: The SQL Tutorial for Data Analysis](https://mode.com/sql-tutorial/introduction-to-sql)
    3. [W3Schools](https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_columns)
    4. [Oracle-Base](https://oracle-base.com/articles/misc/rank-dense-rank-first-last-analytic-functions)
    5. [sqlitetutorial](https://www.sqlitetutorial.net/sqlite-window-functions/sqlite-row_number/) 
    6. [SQLite](https://www.sqlite.org/index.html)
    
    
### Objectives
  - Understanding of UNION and UNION ALL.
  - JOIN: self join and multiple key joins.
  - SQL data types:
      - Dealing with Strings.
      - Dealing with TimeStamp.
      - Dealing with Missing values.
  - Subqueries 
  - 3 Window functions


## Quick Review of last session where the objectives were:
---
  - Define basic characteristics of RDBMS
  - Understand basic relational database terminology
    - Indice types
    - Columns / Fields
    - Relationships (one-to-one, many-to-one)
  - Basic SQL syntax
    - `SELECT`
    - `WHERE` conditionals
    - `GROUP BY` aggregation + `HAVING` conditional filtering
    - `ORDER BY` sorting
    - `LIMIT` limiting results
    - `JOIN` relational joining of tables
    
- White board => 

## SQL - `DISTINCT` & `COUNT`
---
- There is a big difference when one usese `DISTINCT`, `COUNT`
    
    ```sql
    select sum(case when Company is null then 1 else 0 end) as null_val,
    count(Company) as non_null, count(*)
    from Customer c2;```

## SQL - `UNION`
---
- Merges data from two queries by stacking results on top of each other.
- Must have same number of columns and corresponding `data types`.
- Duplicate results are removed by default.
- `UNION ALL` will include duplicates.
- When using `UNION` the rows from the second dataset that are the same are dropped.
- Use `UNION` if you want to append distinct values.

```sql
select 'Customer' as dataset_name,
Country, count(Country) number_people
from Customer 
where Country like 'France'
group by 1 
union ALL
select 'Employee' as dataset_name,
Country, count(Country) number_people
from Employee e
where Country like 'Canada'
group by 1;
```

## SQL - `JOIN with Comparison Operators`
---
- This is super handy for when you are trying to sbset your data after the join.
- Be careful when using this command as it can cause some serious missing values.
```sql
SELECT companies.permalink,
       companies.name,
       companies.status,
       COUNT(investments.investor_permalink) AS investors
  FROM tutorial.crunchbase_companies companies
  LEFT JOIN tutorial.crunchbase_investments_part1 investments
    ON companies.permalink = investments.company_permalink
   AND investments.funded_year > companies.founded_year + 5
 GROUP BY 1,2, 3
```
- Example from Mode Analytics.

## SQL - `JOIN on Multiple Keys, Self JOINS`
---
- The ide here is to increase accuracy of results... but you can hurt the performance of your join. TRUST ME ON THIS IT WILL TAKE TOOOOO LONGGG to get an output back to you. 
- This will be the case when the tables you are joining have more than 1 column that relates them to.
- I can see

```sql
SELECT companies.permalink,
       companies.name,
       investments.company_name,
       investments.company_permalink
  FROM tutorial.crunchbase_companies companies
  LEFT JOIN tutorial.crunchbase_investments_part1 investments
    ON companies.permalink = investments.company_permalink
   AND companies.name = investments.company_name
   ```
 - Example from Mode Analytics.
 
- Self join => They are super helpful specially when you want to extract/combine information that is belongs to different entities but are somehow related...

- List of customers and employee names whose CustomerId is greater or equal than the SupportRepId. Constrain to customers who are from France, Brazil or Canada.

```sql
select cusmer_1.FirstName as cst_name, cusmer_1.LastName as cst_lastname, cusmer_1.Country, 
        cusmer_1.CustomerId, cusmer_1.SupportRepId, empl_1.LastName as empl_lastname, 
        empl_1.FirstName as empl_firstname
from Customer as cusmer_1
join Customer as cusmer_2 
    on cusmer_1.CustomerId = cusmer_2.CustomerId
join Employee as empl_1
    on cusmer_2.SupportRepId = empl_1.EmployeeId
where cusmer_1.CustomerId >= cusmer_2.SupportRepId 
and cusmer_1.Country in ('France', 'Brazil', 'Canada');
```

## SQL - `Data types`
---
- Just as we have seen in Python, SQL also has its own ways to store data. 
- Keep in mind that each SQL flavor has its own constrains on how to store data.
https://www.sqlite.org/datatype3.html
- The most common data types are (from Mode Analytics):


| Data            | Type                                             | How is stored                   |
|:---------------------|:------------------------------------------------------|:-------------------------------|
|String | VARCHAR(1024) or any other value                         | Any characters, with a maximum field length of 1024 characters or any other number.                |
| Date/Time   | TIMESTAMP    | Stores year, month, day, hour, minute and second values as YYYY-MM-DD hh:mm:ss. |
| Number | DOUBLE PRECISION | Numerical, with up to 17 significant digits decimal precision.|
| Boolean              | BOOLEAN                              | Only TRUE or FALSE values.         |

- Note: SQLite does not have a separate Boolean storage class. Instead, Boolean values are stored as integers 0 (false) and 1 (true).



## SQL - `Data types`
---
- TIMESTAMP is probably the one you will have to deal regularly.
- The way how to deal with this data type is unique for every SQL flavor and of course you can cast the output to an integer!!
    - https://www.sqlite.org/lang_datefunc.html
- What is the age of employees when they were hired? 

```sql
select LastName, FirstName, BirthDate, HireDate,
    cast(strftime(HireDate)- strftime(BirthDate)  as int) as_years_at_hire
from Employee e;

```

## SQL - `Dealing with Strings`
---
- SQL is notorious for dealing with TIMESTAMP and string values. 
- String values are notorious for having missing characters at the beginning or end of the column. 
- Often we have entries with lower and upper values that can create problems.
- SQLITE has the following commands to deal with this:

```sql
lower(X)

The lower(X) function returns a copy of string X with all ASCII characters converted to lower case. The default built-in lower() function works for ASCII characters only. To do case conversions on non-ASCII characters, load the ICU extension.

ltrim(X)
ltrim(X,Y)

The ltrim(X,Y) function returns a string formed by removing any and all characters that appear in Y from the left side of X. If the Y argument is omitted, ltrim(X) removes spaces from the left side of X.

rtrim(X)
rtrim(X,Y)

The rtrim(X,Y) function returns a string formed by removing any and all characters that appear in Y from the right side of X. If the Y argument is omitted, rtrim(X) removes spaces from the right side of X.

trim(X)
trim(X,Y)

The trim(X,Y) function returns a string formed by removing any and all characters that appear in Y from both ends of X. If the Y argument is omitted, trim(X) removes spaces from both ends of X.


substr(X,Y,Z)
substr(X,Y)

The substr(X,Y,Z) function returns a substring of input string X that begins with the Y-th character and which is Z characters long. If Z is omitted then substr(X,Y) returns all characters through the end of the string X beginning with the Y-th. The left-most character of X is number 1. If Y is negative then the first character of the substring is found by counting from the right rather than the left. If Z is negative then the abs(Z) characters preceding the Y-th character are returned. If X is a string then characters indices refer to actual UTF-8 characters. If X is a BLOB then the indices refer to bytes.
```
from https://www.sqlite.org/lang_corefunc.html

```sql
select BillingAddress, LOWER(BillingAddress), ltrim(BillingAddress, 82), 
rtrim(lower(BillingAddress), 'street'), SUBSTR(BillingAddress, 5, LENGTH(BillingAddress))
from Invoice i2 ;
```

## SQL - `Missing values`
---
- `COALESCE` comes to the rescue!!!... specially if you want to include in your count null values

```sql
select Company,COALESCE(Company, 'no company')
from Customer;


select count(Company) as ct_1, COUNT(COALESCE(Company, 0)) as ct_2
from Customer;

```



## SQL - `Finding the location of a substring  and concatenation`
---
- The SQLite INSTR searches a substring in a string and returns an integer that indicates the position of the substring, which is the first character of the substring.

```sql
select City, INSTR(City, 'Pau') as test
from Customer c;

select City, INSTR(City, 'Pra') as test
from Customer c;
```

- Where concatenation is handy is when we can combine column content to generate id's, names, etc.

```sql
select City, FirstName, (FirstName ||', '|| LastName||' from: '|| City)
from Customer c;
```

## SQL - `Extracting time components from your tables`
---
- I can't stress the importance of dealing with time in SQL. As I said before each SQL flavor has its own ways to do this. SQL lite uses the following:

- If you want to extract the Year Month and Day for the current date, the following SQL can be used:

```sql
SELECT strftime('%Y %m %d','now');
```

- If you want to extract the Hour Minute Second and milliseconds from the current datetime, the following SQL can be used.

```sql
SELECT strftime('%H %M %S %s','now');
```

```sql
select LastName, FirstName, BirthDate,HireDate,
strftime('%Y', HireDate)
from Employee e;

```

## SQL - `SubQueries`
---
- Think as a nested query where given the output of the internal query you use this output for your outter query. 

```sql
SELECT *
FROM Customers
WHERE CustomerID IN 
	(
       SELECT CustomerID
	  FROM Orders
       GROUP BY CustomerID
       HAVING Count(*) > 1
      )
ORDER BY Country, City, CustomerName
```

## SQL - `Window Function`
---

- FROM MODE ANALYTICS: A window function performs a calculation across a set of table rows that are somehow related to the current row. 
    - This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. 
    - Behind the scenes, the window function is able to access more than just the current row of the query result.

- This feature is so powerful and can save you significatn amount of time in your analysis.

-```Aggregate and Window```.
<p align="center">
  <img src="../../img/sql_window_1.png" width="800" height="1000">
</p>
*Image from sqlitetutorial.net*

## SQL - `Window Function`
---

-```Aggregate and Window```.
<p align="center">
  <img src="../../img/sql_window_2.png" width="800" height="1000">
</p>
*Image from sqlitetutorial.net*

## SQL - `Window Function`
---


| Name         | Description                                                                                                |
|:--------------|:------------------------------------------------------------------------------------------------------------|
| CUME_DIST    | Compute the cumulative distribution of a value in an ordered set of values.                                |
| DENSE_RANK   | Compute the rank for a row in an ordered set of rows with no gaps in rank values.                          |
| FIRST_VALUE  | Get the value of the first row in a specified window frame.                                                |
| LAG          | Provide access to a row at a given physical offset that comes before the current row                       |
| LAST_VALUE   | Get the value of the last row in a specified window frame.                                                 |
| LEAD         | Provide access to a row at a given physical offset that follows the current row.                           |
| NTH_VALUE    | Return the value of an expression evaluated against the row N of the window frame in the result set.       |
| NTILE        | Divide a result set into a number of buckets as evenly as possible and assign a bucket number to each row. |
| PERCENT_RANK | Calculate the percent rank of each row in an ordered set of rows.                                          |
| RANK         | Assign a rank to each row within the partition of the result set.                                          |
| ROW_NUMBER   | Assign a sequential integer starting from one to each row within the current partition.                    |

## SQL - `Window Function - RANK`
---

- The `RANK()` function assigns a rank to each row in a query’s result set. The rank of a row is calculated by one plus the number of ranks that comes before it.
- The syntax is as follows:

```sql
RANK() OVER (
    PARTITION BY <expression1>[{,<expression2>...}]
    ORDER BY <expression1> [ASC|DESC], [{,<expression1>...}]
)
```
- Where the `PARTITION BY` clause divides the rows of the result set into partitions.
- Where the `ORDER BY` clause specifies the orders of the rows in each a partition.
- Where the ` RANK()` function is applied to each row in each partition and re-initialized when crossing the partition boundary.


## SQL - `Window Function - RANK`
---
-```Window Function - RANK```.
<p align="center">
  <img src="../../img/sql_rank_1.png" width="300" height="600">
</p>
*Image from sqlitetutorial.net*

## SQL - `Window Function - RANK`
---
-```Window Function - RANK```.
<p align="center">
  <img src="../../img/sql_rank_2.png" width="300" height="600">
</p>
*Image from sqlitetutorial.net*


- What is happening here? the third and fourth rows receive the same rank because they have the same value (makes sense). The fifth row gets the rank 5 because the RANK() function skips the rank 4 (hmmmm)!!.


## SQL - `Window Function - RANK`
---
- Let's rank tracks by their lengths:

```sql
SELECT Name, Milliseconds,
    RANK () OVER (ORDER BY Milliseconds DESC) LengthRank 
FROM
    track;
```
- Let's take a look at the output:
    - When we skipped the `PARTITION BY` clause, the `RANK()` function treats the whole result set as a single partition.
    - This means that first the `ORDER BY` clause sorts the tracks by their lengths (Milliseconds column).
    - Then the `RANK()` function is applied to each row in the result set considering the orders of tracks by their lengths.
    

- Let's apply the `PARTITION BY` clause and see the output:

```sql
SELECT Name, Milliseconds, AlbumId,
    RANK () OVER (PARTITION BY AlbumId ORDER BY Milliseconds DESC) LengthRank 
FROM
    track;
```
- Let's take a look at the output:
    - First the `PARTITION BY` clause divides the tracks into albums.
    - Then, the `ORDER BY` clause sorts the tracks by their lengths.
    - Lastly, the `RANK()` function assigns a rank to each track in each album. 
        - If the album changes, the `RANK()` function re-initializes the rank value.

## SQL - `Window Function - RANK and Subqueries`
---
- So now you have a table that has a rank based on LengthRank... but in real applications you need the first second. third.... rank.
- Here is where `Window Functions and Subqueries` work together.
- Assume that you want to grab the fourth longest track in each album:


```sql
SELECT 
    * 
FROM (
    SELECT Name, Milliseconds, AlbumId, RANK () OVER (PARTITION BY AlbumId 
                                                        ORDER BY Milliseconds DESC) LengthRank 
    FROM
        track
) 
WHERE 
    LengthRank = 4;
```

## SQL - `Window Function - RANK vs DENSE_RANK`
---
- The `DENSE_RANK()` function computes the rank of a row in an ordered set of rows and returns the rank as an integer. The ranks are consecutive integers starting from 1. Rows with equal values receive the same rank. ***And rank values are not skipped in case of ties.***
- A main source of mistake is not knowing the difference between the `RANK()` vs `DENSE_RANK()` functions.
- The difference is pretty subtle:
    - `RANK()` gives you the ranking within your ordered partition. Ties are assigned the same rank, with the next ranking(s) skipped. So, if you have 3 items at rank 2, the next rank listed would be ranked 5
    - `DENSE_RANK()` again gives you the ranking within your ordered partition, but the ranks are consecutive. No ranks are skipped if there are ranks with multiple items.
 
- You are not sure yet.... well check this great source from [Oracle](https://oracle-base.com/articles/misc/rank-dense-rank-first-last-analytic-functions)

```sql
SELECT
    Name, Milliseconds, AlbumId,
    DENSE_RANK () OVER ( 
        PARTITION BY AlbumId 
        ORDER BY Milliseconds DESC 
    ) LengthRank
FROM
    track;
```

## SQL - `Window Function - DENSE_RANK`
---
-```Window Function - RANK```.
<p align="center">
  <img src="../../img/sql_dense_rank.png" width="300" height="600">
</p>
*Image from sqlitetutorial.net*


- What is happening here:
    - Rows with the same value receive the same rank.
    - There are no gaps in rank values.


## SQL - `Window Function - ROW_NUMBER`
---
- The `ROW_NUMBER` assigns a sequential integer to each row of a query’s result set. Rows are ordered starting from one based on the order specified by the ORDER BY clause in the window definition.
- The syntax is as follows:

```sql
ROW_NUMBER() OVER (
    [PARTITION BY expression1, expression2,...]
    ORDER BY expression1 [ASC | DESC], expression2,...
)
```
- The `PARTITION BY` clause divides the rows derived from the `FROM` clause into partitions. 
- The `PARTITION BY` clause is **optional**. If one skips the `PARTITION BY`, the `ROW_NUMBER()` will treat the whole result set as a single partition. (same idea as the previous window functions).
- The `ORDER BY` clause specifies the order of the rows in each partition. The `ORDER BY` clause is **mandatory** because the `ROW_NUMBER()` function is order sensitive.
- In the output, each row in each partition is assigned a sequential integer number called row number. The row number is reset for each partition.

## SQL - `Window Function - ROW_NUMBER`
---
- Lets take a look at the output of this code that returns the first name, last name, and country of all customers and it uses the `ROW_NUMBER()` function to add a sequential integer to each customer record:

```sql
SELECT
    ROW_NUMBER () OVER (ORDER BY Country) RowNum,
    FirstName,
    LastName,
    country 
FROM
    customer;
 ```
 
- Lets take a look at the output of this code that assigns a sequential integer to each customer and resets the number when the country of the customer changes: 
 
 
```sql
SELECT
    ROW_NUMBER () OVER (PARTITION BY Country ORDER BY FirstName) RowNum,
    FirstName,
    LastName,
    country 
FROM
    customer;
```

- In the last code:
    - The `PARTITION BY` clause divides the customers by into partitions by country.
    - The `ORDER BY` clause sorts customers in each partition by the first name.
    - The `ROW_NUMBER()` function assigns each row in each partition a sequential integer and resets the number when the country changes.